In [65]:
import pandas as pd
import numpy as np
import zipfile
import textdistance
import re
import pickle
from veryfi import Client
from itertools import chain

In [24]:
#import zipfile
#with zipfile.ZipFile("documents_to_process.zip","r") as zip_ref:
#    zip_ref.extractall("unzippedDocs")

In [4]:
#list_files = ['0001431487.jpg', '0060027768.jpg','0060027778.jpg','0060068160.jpg']


zip_file_path = 'documents_to_process.zip'

client_id = ''
client_secret = ''
username = ''
api_key = ''

veryfi_client = Client(client_id, client_secret, username, api_key)
response = veryfi_client.process_document(zip_file_path)

In [25]:
#responce2=[]
#for i in list_files:
#    responce2.append(veryfi_client.process_document('unzippedDocs/'+i))

In [17]:
docs=response['ocr_text'].split('\x0c')

In [40]:
counter=0
for i in docs:
    t=i.split('\n')
    t=[j.split('\t') for j in t]
    docs[counter]=t
    counter+=1

In [151]:
#with open('processedDocs.pkl','wb') as file:
#    pickle.dump(docs,file)

In [246]:
with open('processedDocs.pkl', 'rb') as f:
    docs = pickle.load(f)

In [247]:
#sup1: Kw1 in invoice
#sup2: name in caps and without numbers

def findKwLoc(kw,doc,sensibility=3):
    pi=0
    for i in doc:

        pj=0
        for j in i:
            dst=textdistance.damerau_levenshtein.distance(kw,j)
            
            if dst<sensibility:
                location=(pi,pj)
                return(location)
            pj+=1
        pi+=1
    

In [248]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

def has_lowercase(inputString):
    return bool(re.search(r'[a-z]', inputString))


In [249]:
kw1='BILLING INSTRUCTIONS MAL YOUR INVOICES IN DUPLICATE TO CONSIGNEE'

In [250]:
vendor_names=[]
for i in docs:
    poss=findKwLoc(kw1,i,3)
    if poss==None:
        vendor_names.append(-1)
        continue
    options=[j[0] for j in i[0:poss[0]]]
    boli=[not (has_numbers(j) or has_lowercase(j) or j=='') for j in options]
    options=list(np.array(options)[boli])
    vendor_names.append(' '.join(options))
    
    
    

In [251]:
kw2='AS SHOWN BELOW'
kw3='SHIPMENT TO ARRIVE NOT LATER THAN'

In [252]:
#sup3: Kw2 in invoice
#sup4: Kw3 in invoice
#sup5: "TO:" not in shipping details 
#sup6: shipping details has more than 3 lines

shippinngDeets=[]
for i in docs:
    poss1=list(findKwLoc(kw2,i,3))
    poss2=list(findKwLoc(kw3,i,3))
    if poss1==None or poss2==None:
        shippinngDeets.append(-1)
        continue
    poss1[0]=poss1[0]+2
    poss2[0]=poss2[0]-1
    
    shippinngDeets.append(i[poss1[0]:poss2[0]])

In [253]:
# use TO: to further delimit the search
for i in range(len(shippinngDeets)):
    frst3=shippinngDeets[i][0:3]
    frst3=[' '.join(j) for j in frst3]
    counter=0
    index=-1
    for j in frst3:
        if 'TO:' in j:
            index=counter
        counter+=1
    if index==-1:
        continue
    [shippinngDeets[i].pop(0) for k in range(index+1)]

In [254]:
#sup7: Shipping details are not included if kw 'ATTACH' is included in the description
#
kw4='ATTACH'

In [255]:
def isattached(doc):
    for i in doc:
        for j in i:
            if kw4 in j.upper():
                return(True)
    return(False)

In [256]:
#sup 8: shipping details has at least one line

def breakShipping(doc):
    attached=isattached(doc)
    counter=0
    bills=[]
    ships=[]
    for i in doc:
        if sum([len(j) for j in i])<2:
            continue
        
        if '' in i:
            bill=i[0:i.index('')]
            ship=i[i.index(''):]
        elif counter!=0 and not attached:
            if len(i)%2==0:
                bill=i[0:int(len(i)/2)]
                ship=i[int(len(i)/2):]
            else:
                bill=i[0:int((len(i)-1)/2)]
                ship=i[int((len(i)+1)/2):]
        elif counter==0 and attached:
            ship=i[-1]
            bill=i[0:-1]
        else:
            ship=['']
            bill=i
        ships.append(ship)
        bills.append(bill)
        counter+=1
    return([bills,ships,attached])

In [257]:
def giveshipping(doc):
    breaked=breakShipping(doc)
    billname=' '.join(breaked[0][0])
    billname=billname.replace('  ',' ')
    billadd=list(chain(*breaked[0][1:]))
    billadd=' '.join(billadd)
    billadd=billadd.replace('  ',' ')
    if breaked[2]:
        shipname='ATTACHED'
        shipadd='ATTACHED'
    else:
        shipname=' '.join(breaked[1][0])
        shipname=shipname.replace('  ',' ')
        shipadd=list(chain(*breaked[1][1:]))
        shipadd=' '.join(shipadd)
        shipadd=shipadd.replace('  ',' ')
    return(billname, billadd, shipname, shipadd)

In [258]:
bill_to_names=[]
bill_to_address=[]
ship_to_names=[]
ship_to_address=[]

for i in shippinngDeets:
    proc=giveshipping(i)
    bill_to_names.append(proc[0])
    bill_to_address.append(proc[1])
    ship_to_names.append(proc[2])
    ship_to_address.append(proc[3])

In [259]:
def undobbleSpace(x):
    y=x
    while '  ' in y:
        y=y.replace('  ',' ')
    return(y)

def findKwCon(kw,doc,sensibility=3):
    counter=0
    for i in doc:
        dst=textdistance.damerau_levenshtein.distance(kw,i)
        if dst<sensibility:
            return(counter)
        counter+=1
        
def beforeThirdSpace(x):
    spaces=[i.start() for i in re.finditer(r" ",x)]
    if len(spaces)>=3:
        return(x[0:spaces[2]])
    else:
        return(x)

In [260]:
kw5='QUANTITY CODE NO. DESCRIPTION PRICE'
kw6='ACCOUNTING CHARGE NO.'

In [261]:
def startitem(doc):
    connected=[undobbleSpace(' '.join(i[0:])) for i in doc]
    pi=findKwCon(kw5,connected,3)+1
    connected=[beforeThirdSpace(i) for i in connected]
    pf=findKwCon(kw6,connected,3)-1
    connected=doc[pi:pf]
    count1=0
    found=False
    for i in connected:
        if found:
            break
        count2=0
        for j in i:
            if j.replace(',','').replace('.','').isnumeric():
                found=True
                break
            count2+=1
        count1+=1
    if found:
        number=connected[count1-1][count2]
        del connected[count1-1][count2]
    else:
        number=-1
    found=False
    count1=0
    for i in connected:
        if found:
            break
        count2=0
        for j in i:
            if '$' in j:
                found=True
                break
            count2+=1
        count1+=1
    if found:
        price=connected[count1-1][count2]
        del connected[count1-1][count2]
    else:
        price=-1
    desc=[' '.join(i) for i in connected]
    desc=undobbleSpace(' '.join(desc))
    return(number,price,desc)

In [262]:
startitem(docs[0])

('6,500',
 '$1.45 each',
 ' Keyrings to be used in connection with the PALL MALL Quality Collection Self-Liquidating Promotion. Keyrings, as per sample submitted and approved; Gold Finish, engraved and color filled in red PALL MALL. Keyrings are to be packaged in a Cardboard Gift Box. This Purchase Order is subject to and acceptance hereof expressly limited to the terms and conditions on the face and reverse side hereof and on the Specification Sheets. attached hereto and expressly made a part hereof.')